In [70]:
import sys
import os
#from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import math
import numpy as np

%load_ext autoreload
%autoreload 2
#import skimage.measure

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This is data loading stuff..

In [71]:
#mnist = input_data.read_data_sets('./mnist_data')

#model_name = "mnist5_blue2"
# honeslty idk what this name was for lol
#file_name = "mnist5.pb"

# Pick data set:

In [120]:
#data_name = "balanced"
#data_name = "blue2-80"

#data_name = "balanced-100"
data_name = "blue2-100"


#network_arch = [50, 50, 50, 50]

In [121]:
data_train = np.load(data_name+'_train.npz')
data_trainX = data_train['X']
data_trainY = data_train['y']

data_test = np.load(data_name+'_test.npz')
data_testX = data_test['X']
data_testY = data_test['y']



In [122]:
data_trainX = np.reshape(data_trainX, (-1, 28,28,3))
data_testX = np.reshape(data_testX, (-1, 28,28,3))
data_trainX.shape

(60000, 28, 28, 3)

This is debugging stuff...

In [97]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def reshape_batch(x):
    modified_x = []
    for image in x:
        i = np.reshape(image, (28, 28))
        modified_x.append(np.reshape(skimage.measure.block_reduce(i, (2, 2), np.max), (196)))
    modified_x = np.array(modified_x)
    return modified_x

In [123]:
model_name = "network_"+data_name
network_arch = "3x20"
model_name += "_"+network_arch

In [124]:
model_name

'network_blue2-100_3x20'

This is really what you need to use to train. We will use the models stored in the `models.py` file, not the debugging shit above. 

In [125]:
import models
tf.reset_default_graph()
x_in, y, cross_entropy, y_ = models.model_fn([20,20,20])
#x_in, y, cross_entropy, y_, _ = models.model_fn_oldold()
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)


## Training loop is here

Assumes `data_trainX` and `data_trainY` are already loaded, and `model_name` is set. 

The network checkpoint (i.e. the weights) are saved to `'models/'+model_name+'.ckpt'` 

In [126]:
epochs = 10
batch_size = 32

N = data_trainX.shape[0]
batch_amount = epochs*N//batch_size

with tf.Session() as sess:
    epoch_in = 1
    tf.global_variables_initializer().run()
    for i in range(batch_amount):
        if (i*batch_size) % N == 0:
            print(epoch_in)
            #print('loss> {}'.format(sess.run(cross_entropy, feed_dict={x_in: batch_xs, y_: batch_ys})))
            #graph = tf.get_default_graph()
            #print(sess.run(graph.get_tensor_by_name('fc1/weight:0')))
            epoch_in += 1
        #batch_xs, batch_ys = mnist.train.next_batch(50)
        batch_xs = data_trainX[range(i*batch_size % N, i*batch_size % N + batch_size)]
        batch_ys = data_trainY[range(i*batch_size % N, i*batch_size % N + batch_size)]
        # modified_batch_xs = reshape_batch(batch_xs)
        sess.run(train_step, feed_dict={x_in: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y, 1), y_)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # modified_test_x = reshape_batch(test_x)

    print(sess.run(accuracy, feed_dict={x_in: data_testX, y_: data_testY}))
    
    print("before checkpoint")
    
    # Save the weights of the trained network. 
    saver = tf.train.Saver()
    save_path = saver.save(sess, 'models/'+model_name+'.ckpt')
  
    ## This is Chris's weird protobuf convert parse save idkwtfsmh thing...
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['output_op'])
    with tf.gfile.GFile('models/' + model_name+'.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())
    

1
2
3
4
5
6
7
8
9
10
0.9217
before checkpoint
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


In [127]:
model_name

'network_blue2-100_3x20'

- balanced-100_5x50 acc: 0.9421
- blue2-100_5x50 acc: 0.9527

- balanced-100_3x50 acc: 0.9406
- blue2-100_3x50 acc: 0.955

- balanced-100_3x20 acc: 0.887
- blue2-100_3x20 acc: 0.9217

- balanced_5x50 acc: 0.9354
- blue2-80_5x50 acc: 0.9406


- balanced_3x50 acc: 0.9379
- blue2-80_3x50:acc: 0.942


- balanced_3x20 acc: 0.9046
- blue2-80_3x20 acc: 0.907


